In [1]:
import time
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import os

In [8]:
ckey = 'AudDS2qWKad8bnQhytqozVV9p'
consumer_secret = 'nGApgLkYWN5qjVnDk1LFY307XbKk3x6LP2z8TmdMBaqvS1XLdo'
access_token_key = '3296532819-xSeTwH2AO8SExtRQcg3p7KYPVJYY6F4wxzbbhCT'
access_token_secret = '3XnFOBl0CBZ2uCESsKxKyoZee9eBgf1jfRvxW9nYxpeeE'
 
 
start_time = time.time() #grabs the system time

keyword_list = ['telefonica'] #track list

In [9]:
import tweepy
import sys
import pika
import json
import time
import pprint
pp = pprint.PrettyPrinter(indent=1)

#get your own twitter credentials at dev.twitter.com
consumer_key = ckey
consumer_secret = consumer_secret
access_token = access_token_key
access_token_secret = access_token_secret

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

class CustomStreamListener(tweepy.StreamListener):
    def __init__(self, api):
        self.api = api
        super(tweepy.StreamListener, self).__init__()

        #setup rabbitMQ Connection
        connection = pika.BlockingConnection(
            pika.ConnectionParameters(host='localhost')
        )
        self.channel = connection.channel()

        #set max queue size
        args = {"x-max-length": 2000}

        self.channel.queue_declare(queue='twitter_topic_feed', arguments=args)

    def on_status(self, status):
        print status.text
        print 'Localización: ', status.geo
        print 'Source: ' , status.source
        print 'Created at:' , status.created_at
        print "Lang: ", status.lang
        
        print
        
        print

        data = {}
        data['text'] = status.text
        data['created_at'] = time.mktime(status.created_at.timetuple())
        data['geo'] = status.coordinates
        data['source'] = status.source
 
        
        #queue the tweet
        self.channel.basic_publish(exchange='',
                                    routing_key='twitter_topic_feed',
                                    body=json.dumps(data))

    def on_error(self, status_code):
        print 'Encountered error with status code:', status_code
        print >> sys.stderr, 'Encountered error with status code:', status_code
        return True  # Don't kill the stream

    def on_timeout(self):
        print >> sys.stderr, 'Timeout...'
        return True  # Don't kill the stream


In [ ]:
sapi = tweepy.streaming.Stream(auth, CustomStreamListener(api))
# my keyword today is chelsea as the team just had a big win
sapi.filter(track=keyword_list) 

operator facturare (call center): preluarea telefonica a comenzilor,editarea facturilor si a comenzilor,… http://t.co/wAwTEEoT50
Localización:  None
Source:  dlvr.it
Created at: 2015-07-21 08:40:37
Lang:  ro


Top story: Vozpópuli - Escándalo en la CNMC: un consejero ocultó cobros millona… http://t.co/j4wV7nMuH0, see more http://t.co/no8PBME5pa